# 受限玻尔兹曼机实验
## 1. BernoulliRBM

加载之前预训练好的模型， 参数与下面的设置一样

In [6]:
from rbms import BernoulliRBM
from IPython.display import Image
import imageio
from utils import *

In [7]:
brbm = BernoulliRBM(
    model_path='./logs/brbm_pcd/',
    pcd=True,
    lr_start=2,
    sparsity_target=0.1,
    lr_stop=8,
    n_gibbs_steps=2,
    max_epoch=10,
    sample_h_states=False,
    sample_v_states=True,
    batch_size=32,
    h_sz=256,
    l2=0.01)

实验1： 分别从各时间段采样，查看Markov Chain迁移的情况，图片生成的情况。
1. 用随机做h0的初始化。
2. 用测试样本作为v0的初始化。

每个20个gibbs_step生成一张图片，把生成的图片转化成gif动态图，方便查看和展示。

用随机做h0的初始化的实验结果：

In [10]:
Image(url='./images/sto_pcd.gif')

用测试样本作为v0的初始化的实验结果：

In [11]:
Image(url='./images/val_pcd.gif')

### 实验总结
1. 用随机初始化，还是用测试样本初始化，最终模型都能收敛到平稳分布，说明之前的rbm训练ok。
2. 随着Markov_Chain的转移，模型基本能遍历各个model， 多样性很好。
3. 若同样的配置， 把pcd换成cd算法，会失败，模型不能收敛到平稳分布。

为什么CD-k算法在这样的配置下确不成功尼，首先看下在这样的配置下，用CD-k算法学出来的图片的几个现象：
1. 根据终端打印出来的重构项很低（大概在0.02左右，pcd算法的在0.07左右），这说明马尔科夫链的mix-rate很低。
2. 用随机初始化做生成，基本没法采样出数字，全是噪声，说明模型根本没有收敛。用测试数据做初始化，从马尔科夫链上采样的图片基本没啥变化，同样说明其mix-rate很低。

在以上分析的基础上，我对模型参数做了几点调整：1）输入加入dropout；2）增加gbbis_step；3）增加max_epoch。

In [12]:
brbm_cd = BernoulliRBM(
    model_path='./logs/brbm_cd/',
    pcd=False,
    drop_probs=0.2,
    lr_start=2,
    sparsity_target=0.1,
    lr_stop=28,
    n_gibbs_steps=10,
    max_epoch=30,
    sample_h_states=True,
    sample_v_states=True,
    batch_size=10,
    h_sz=256,
    l2=0.001)

直接贴结果：

用随机做h0的初始化的实验结果：

In [13]:
Image(url='./images/sto_cd.gif')

In [14]:
Image(url='./images/val_cd.gif')

用测试样本作为v0的初始化的实验结果：

*总结：* CD-k算法的mix-rate很慢，上面的3个方式都能帮助模型收敛，但从结果上看效果能没法匹敌pcd算法，最后模型收敛的仍然不好